In [1]:
# This is needed for some builds
import rosetta
import pyrosetta
pyrosetta.init('-ignore_unrecognized_res T -ignore_waters T -preserve_header T -prevent_repacking T')

Found rosetta database at: /home/stephanie/anaconda2/envs/DebugPyrosetta/lib/python2.7/site-packages/pyrosetta-4.0-py2.7.egg/database; using it....
PyRosetta-4 2016 [Rosetta 2016 unknown:9ea8e5e15e7c35838a32b8089ca8351ff540888c 2016-12-16 10:52:45 -0500] retrieved from: git@github.com:RosettaCommons/main.git
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.



In [14]:
def tmalign( pose, ref_pose ):
    
    print 'Running tmalign on poses'
    tm = rosetta.protocols.hybridization.TMalign()
    tm.apply(pose, ref_pose)
    longest = max(pose.total_residue()+1, ref_pose.total_residue()+1)
    
    print 'TMScore = %s ' %tm.TMscore(longest)
    atommap =  rosetta.core.id.AtomID_Map_core_id_AtomID_t()
    rosetta.core.pose.initialize_atomid_map_AtomID( atommap, pose )
    tm.alignment2AtomMap( pose, ref_pose, atommap )
    aln_cutoff = rosetta.utility.vector1_double()
    for i in [2.0,1.5,1.0,0.5]:
        aln_cutoff.append(i)
    min_coverage = .2
    rosetta.protocols.hybridization.partial_align(pose,ref_pose, atommap, True, aln_cutoff, min_coverage)

In [3]:
# Read in a pose and setup the pymol observer, you have to score it to get it into pymol

p = pyrosetta.pose_from_file('2VBF_cleaned.pdb')
pm = pyrosetta.PyMolMover()
pm.keep_history(True)
pm.apply(p)
# sfxn = pyrosetta.get_fa_scorefxn()
# sfxn(p) 

In [6]:
## Take a look at your protein and the wierd chain endings
print p
print p.conformation().chain_endings()

# Fix the weird chain endings (so that later, we can create a nice foldtree)
p.update_pose_chains_from_pdb_chains()
print p.chain_sequence(1)
print p.conformation().chain_endings()
n_jumps = p.num_jump()
seq1 = p.chain_sequence(1)

PDB file name: 2VBF_cleaned.pdb
Total residues:1082
Sequence: MYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISREDMKWIGNANELNASYMADGYARTKKAAAFLTTFGVGELSAINGLAGSYAENLPVVEIVGSPTSKVQNDGKFVHHTLADGDFKHFMKMHEPVTAARTLLTAENATYEIDRVLSQLLKERKPVYINLPVDVAAAKAEKPALSLENTTEQVILSKIEESLKNAQKPVVIAGHEVISFGLEKTVTQFVSETKLPITTLNFGKSAVDESLPSFLGIYNGKLSEISLKNFVESADFILMLGVKLTDSSTGAFTHHLDENKMISLNIDEGIIFNKVVEDFDFRAVVSSLSELKGIEYEGQYIDKQYEEFIPSSAPLSQDRLWQAVESLTQSNETIVAEQGTSFFGASTIFLKSNSRFIGQPLWGSIGYTFPAALGSQIADKESRHLLFIGDGSLQLTVQELGLSIREKLNPICFIINNDGYTVEREIHGPTQSYNDIPMWNYSKLPETFGATEDRVVSKIVRTENEFVSVMKEAQADVNRMYWIELVLEKEDAPKLLKKMGKLFAEQNKMYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISREDMKWIGNANELNASYMADGYARTKKAAAFLTTFGVGELSAINGLAGSYAENLPVVEIVGSPTSKVQNDGKFVHHTLADGDFKHFMKMHEPVTAARTLLTAENATYEIDRVLSQLLKERKPVYINLPVDVAAAKAEKPALSLENTTEQVILSKIEESLKNAQKPVVIAGHEVISFGLEKTVTQFVSETKLPITTLNFGKSAVDESLPSFLGIYNGKLSEISLKNFVESADFILMLGVKLTDSSTGAFTHHLDENKMISLNIDEGIIFNKVVEDFDFRAVVSSLSELKGIEYEGQYIDKQYEEFIPSSAPLSQDRLWQAVESLTQSNETIVAEQGTSFFGASTIFLKSNSRFIGQPLWGS

In [8]:
# Now, since the protein is in the right place, we just make a copy of the chain A
ChainAcopy = rosetta.core.pose.Pose(p,1,len(seq1))
ChainBcopy = rosetta.core.pose.Pose(p,len(seq1)+1,p.total_residue())

In [24]:
ChainA = rosetta.core.pose.Pose(p,1,len(seq1))
## Make third copy of chainA
## align copyChainA onto chainB
## Then turn the chain ID to Chain B on the copy
## then append chainb to chain A
## then dump pdb - then run make symm def file

In [15]:
tmalign(ChainAcopy,ChainBcopy)

Running tmalign on poses
TMScore = 0.995005610329 


In [16]:
pm.apply(ChainAcopy)

In [21]:
for i in range(1,len(seq1)+1):
    ChainAcopy.pdb_info().chain(i,'B')
ChainAcopy.pdb_info().obsolete(False)
ChainAcopy.dump_pdb('isthischainB.pdb')

True

In [23]:
rosetta.core.pose.append_pose_to_pose(ChainA,ChainAcopy)
ChainA.pdb_info().obsolete(False)
ChainA.dump_pdb('thismightworksymm.pdb')

True

In [27]:
rosetta.core.pose.append_subpose_to_pose(ChainA, ChainAcopy, 1, len(seq1), True)
ChainA.dump_pdb('thismightworksymm.pdb')

True